<a href="https://colab.research.google.com/github/pravsels/DistilLM/blob/main/issues_dataset_git_repo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers[sentencepiece] requests
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [44]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
import requests
from datasets import load_dataset
import os
import subprocess

In [40]:
# put in personal access token (PAT) here
GIT_TOKEN = ''
headers = {'Authorization': f'token {GIT_TOKEN}'}

hf_username='pravsels'
owner="3b1b"
repo="manim"
branch="main"

In [ ]:
# base_url = "https://api.github.com/repos"
# owner="3b1b"
# repo="manim"
# per_page = 100
# page=0

# query = f"issues?page={page}&per_page={per_page}&state=closed"
# issues = requests.get(f"{base_url}/{owner}/{repo}/{query}")
# response = requests.get(f"{base_url}/{owner}/{repo}/{query}")

In [ ]:
def fetch_issues(
    owner="3b1b",
    repo="manim",
    issue_state='closed',
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state={issue_state}"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())
        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [ ]:
def get_comments(issue_number):
  url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}/comments"
  response = requests.get(url, headers=headers)
  return [r['body'] for r in response.json()]

get_comments(11)

In [ ]:
# fetch issues into df, then load as dataset
fetch_issues()

issues_dataset = load_dataset("json",
                              data_files="manim-issues.jsonl",
                              split="train")
issues_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'draft', 'pull_request', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason'],
    num_rows: 1593
})

In [ ]:
# adding column to delineate issues and pull requests
issues_dataset = issues_dataset.map(
    lambda x: {'is_pull_request': False if x['pull_request'] is None else True}
)

In [ ]:
# adding a comments column
issues_with_comments = issues_dataset.map(
    lambda x: {'comments': get_comments(x['number'])}
)

Map:   0%|          | 0/1593 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
issues_with_comments.push_to_hub(f"{hf_username}/{repo}_{owner}_issues")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/pravsels/manim_3b1b_issues/commit/95e4273373f1d9d3e5a4bfc8594fd3cb6a5bacdf', commit_message='Upload dataset', commit_description='', oid='95e4273373f1d9d3e5a4bfc8594fd3cb6a5bacdf', pr_url=None, pr_revision=None, pr_num=None)

In [46]:
def clone_and_process_repo(repo_url, local_dir, process_function):
    # Clone the repository
    subprocess.run(["git", "clone", repo_url, local_dir], check=True)

    for root, dirs, files in os.walk(local_dir):
        for name in files:
            file_path = Path(root) / name
            process_function(file_path)

def example_process_function(file_path):
    print(f"Processing file: {file_path}")

# Example usage:
repo_url = f"https://github.com/{owner}/{repo}.git"
local_dir = "manim_repo"

if not Path(local_dir).exists():
    clone_and_process_repo(repo_url,
                           local_dir,
                           example_process_function)
else:
    print(f"Directory '{local_dir}' already exists. Please remove it or choose a different directory.")


Directory 'manim_repo' already exists. Please remove it or choose a different directory.
